In [64]:
import math
from datetime import datetime

import pandas as pd
from pandas import DataFrame
import numpy as np

### Load Data

In [65]:
rating_df = pd.read_csv("./ml-25m/ratings.csv")
movie_df = pd.read_csv("./ml-25m/movies.csv")
genre_df = pd.read_csv("./ml-25m/genome-scores.csv")

In [66]:
rating_df.isnull().sum(), movie_df.isnull().sum(), genre_df.isnull().sum()

(userId       0
 movieId      0
 rating       0
 timestamp    0
 dtype: int64,
 movieId    0
 title      0
 genres     0
 dtype: int64,
 movieId      0
 tagId        0
 relevance    0
 dtype: int64)

### 결측치 확인

In [67]:
rating_df = rating_df.sort_values(['timestamp', 'userId'])

In [68]:
train_df = rating_df[(rating_df['timestamp'] >= 1104505203) & (rating_df['timestamp'] <= 1230735592)]
test_df = rating_df[rating_df['timestamp'] >= 1230735600]

In [69]:
print("Train: ", len(train_df) / (len(train_df) + len(test_df)) * 100) # train data is too small...
print("Test: ", len(test_df) / (len(train_df) + len(test_df)) * 100)

Train:  28.629003411388805
Test:  71.37099658861119


### Pivot Table

In [70]:
# train_df = train_df.set_index("userId")

# Construct rating Matrix
train_df = train_df.groupby(["userId", "movieId"])['rating'].sum().unstack()

# fill 0 value in NaN
train_df = train_df.fillna(0)

train_df.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,64957,64969,64976,64983,64986,64990,64993,64997,65011,65025
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
train_data = train_df.values

### matrix decomposition

In [72]:
# U, s, VT = np.linalg.svd(new_train, compute_uv = True)

In [73]:
# r = s.shape[0] - sum(lx < 1.e-8 for lx in s)

In [74]:
# svd를 하고 다시 복원을 하면 빈 값들이 채워진다. 이것을 모델로 만들어서 사용?
# SVD 자체를 numpy로 구현하고 Train data 사용 모델링, 피팅 이후 Test data로 평가...?
# SVD 를 사용 
# 모든 형태의 Matrix에 적용 가능
# U: AAT 의 고유벡터
# V: ATA 의 고유벡터

### r = qTp구현

- Singular Value Decomposition U, s, Vt = svd(new_train)
- new_train data is too sparse
- U: AAT 의 고유벡터
- s: A의 특이값들을 대각항으로 가지는 대각행렬
- VT: ATA의 고유벡터
- s는 eigenvalue에 sqrt값을 취한 값으로 scaling으로 볼수 있음

In [75]:
lr = 0.01
lm = 0.001
epoch = 10

In [76]:
U, s, VT = np.linalg.svd(train_data, compute_uv=True)

In [77]:
r = s.shape[0] - sum(lx < 1.e-8 for lx in s) # s.shape[0] - sum(s < 1.e-8)

sqrt_s = np.array([math.sqrt(a) for a in s]) # np.sqrt(s)

In [78]:
pu = U[:, :r] @ np.diag(sqrt_s[:r])
qi = np.diag(sqrt_s[:r]) @ VT[:r, :]

In [79]:
for _ in range(1, epoch+1):
    R_hat = pu @ qi
    
    err = (train_data - R_hat)**2
    
    new_qi = qi + (lr * ((pu.T @ err) - (lm * qi)))
    new_pu = pu + (lr * ((err @ qi.T) - (lm * pu)))
    
    qi, pu = new_qi, new_pu
    print("epoch : {} / datetime : {} / loss : {}".format(_, datetime.now(), err.sum()))

epoch : 1 / datetime : 2020-09-08 17:20:52.748381 / loss : 2.4571335762598183e-21
epoch : 2 / datetime : 2020-09-08 17:22:33.722755 / loss : 0.024134378354751043
epoch : 3 / datetime : 2020-09-08 17:24:11.701134 / loss : 0.09636490511179578
epoch : 4 / datetime : 2020-09-08 17:25:51.035957 / loss : 0.21592755122088877
epoch : 5 / datetime : 2020-09-08 17:27:28.872396 / loss : 0.38141903381029885
epoch : 6 / datetime : 2020-09-08 17:29:09.964341 / loss : 0.5908709071632912
epoch : 7 / datetime : 2020-09-08 17:30:50.621690 / loss : 0.8418544756489272
epoch : 8 / datetime : 2020-09-08 17:32:33.408689 / loss : 1.131606119496684
epoch : 9 / datetime : 2020-09-08 17:34:09.233265 / loss : 1.4571612698716248
epoch : 10 / datetime : 2020-09-08 17:35:49.500872 / loss : 1.815485721267283


In [80]:
# pu * qi => rating

# numpy로 데이터 읽어오기

In [2]:
data = np.genfromtxt("./ml-25m/ratings.csv", delimiter=",", names=True)

In [3]:
len(data), data, data.shape

(25000095,
 array([(1.00000e+00,   296., 5. , 1.14788004e+09),
        (1.00000e+00,   306., 3.5, 1.14786882e+09),
        (1.00000e+00,   307., 5. , 1.14786883e+09), ...,
        (1.62541e+05, 56176., 2. , 1.24095070e+09),
        (1.62541e+05, 58559., 4. , 1.24095343e+09),
        (1.62541e+05, 63876., 5. , 1.24095252e+09)],
       dtype=[('userId', '<f8'), ('movieId', '<f8'), ('rating', '<f8'), ('timestamp', '<f8')]),
 (25000095,))

In [4]:
# timestamp 기준으로 나누기
train_data = np.array([list(i) for i in data if 1104505203 <= i[3] <= 1230735592])
test_data = np.array([list(i) for i in data if i[3] >= 1230735600])
print(train_data)

[[1.00000000e+00 2.96000000e+02 5.00000000e+00 1.14788004e+09]
 [1.00000000e+00 3.06000000e+02 3.50000000e+00 1.14786882e+09]
 [1.00000000e+00 3.07000000e+02 5.00000000e+00 1.14786883e+09]
 ...
 [1.62521000e+05 5.96150000e+04 3.50000000e+00 1.21933944e+09]
 [1.62521000e+05 5.97250000e+04 4.00000000e+00 1.21933939e+09]
 [1.62521000e+05 6.00740000e+04 4.00000000e+00 1.21933941e+09]]


In [5]:
user_id = np.unique(np.array([i[0] for i in train_data]))  # (27915,))
movie_id = np.unique(np.array([i[1] for i in train_data]))  # (10533,))
rating = np.array([i[2] for i in train_data])

In [55]:
from collections import defaultdict

rating_dict = defaultdict(dict)
for i in train_data:
    rating_dict[i[0]][i[1]] = i[2]

In [56]:
# movie_id의 id가 각 키의 values에 없다면, movie-id를 추가하고 값을 0으로 넣는다. 이 과정을 거치면 fillna(0)과 같은 효과
# 그리고 추가된 rating_dict에서 key를 row, values flatten하게 쌓으면된다.
# user_id (27915,))
# movie_id (10533,))
for i in range(len(user_id)):
    for j in range(len(movie_id)):
        if movie_id[j] not in rating_dict[user_id[i]].keys():
            rating_dict[user_id[i]][movie_id[j]] = 0.0
#         else:
#             pass

In [58]:
# rating_dict의 키(user_id)마다 movie_id를 기준으로 내림차순 정렬, matrix로 재구성
rating_matrix = []
for i in range(len(rating_dict)):
    sorted(rating_dict[user_id[i]])

AttributeError: 'dict_keys' object has no attribute 'item'